In [ ]:
train = '/mnt/d/autocls_e4/trainref/cosc21_train_lvl_1.tif'

feat = '/home/jasp/autocls/feat/feat_allbands.tif'

iterations = 100

proportion = 30

classes = {
    1 : 'Artificial',
    2 : 'Agriculture',
    3 : 'Tree Cover',
    4 : 'Shrubland',
    5 : 'Herbaceous',
    6 : 'Non-vegetated',
    7 : 'Welands',
    8 : 'Water'
}

classfier = 'LogisticRegression'

out = '/home/jasp/autocls/exp4_lg/'

In [ ]:
import os
from osgeo import gdal
import numpy as np
import pandas as pd
from joblib import load

from glass.pys.oss import mkdir
from glass.smp import split_rst_radomly
from glass.rst.cls import train_to_mdl, imgcls_from_mdl
from glass.rd.rst import rsts_to_featarray
from glass.rst.confmtx.totbl import confmtx_twrst, confmtx_fmrst
from glass.wt import obj_to_tbl

In [ ]:
# Create output folders
train_folder = mkdir(os.path.join(out, 'train_folder'), overwrite=True)
valid_folder = mkdir(os.path.join(out, 'valid_raster'), overwrite=True)
model_folder = mkdir(os.path.join(out, 'models'), overwrite=True)
class_folder = mkdir(os.path.join(out, 'class'), overwrite=True)
prob_folder  = mkdir(os.path.join(out, 'probs'), overwrite=True)

In [ ]:
outcomes = []

for i in range(iterations):
    # Split Reference into Train and Validation
    trst, rrst = split_rst_radomly(
        train, proportion,
        os.path.join(train_folder, f'trainrst_{str(i+1)}.tif'),
        os.path.join(valid_folder, f'validrst_{str(i+1)}.tif'),
        min_sample=1000
    )

    # Train Model
    fmodel = train_to_mdl(
        trst, feat,
        os.path.join(model_folder, f'model_{str(i+1)}.bz2'),
        method=classfier, ntrees=1000,
        mxsamples=None
    )

    # Classify and save probabilities
    subprob = mkdir(os.path.join(prob_folder, f'prob_{str(i+1)}'), overwrite=True)
    classrst, fi = imgcls_from_mdl(
        fmodel, feat,
        os.path.join(class_folder, f'class_{str(i+1)}.tif'),
        newmodel=None, probrst=subprob
    )

    outcomes.append({
        'train' : trst,
        'ref'   : rrst,
        'model' : fmodel,
        'class' : classrst,
        'prob'  : subprob
    })

In [ ]:
# Validate

maincols = ['exp', 'total_area', 'overall_accuracy', 'kappa', 'macro-f1', 'weighted-f1']
mtxs, sintese = [], []
expname = []

i = 1
for outcome in outcomes:
    ename = f'exp_{str(i)}'
    mtx = confmtx_fmrst(
        outcome['ref'],
        outcome['class'],
        class_labels=classes
    )

    mtx = mtx.set_index('class')

    mcols = [c for c in mtx.columns.values if c != 'row_area' and c != 'precision']

    if i == 1:
        maincols.extend(mcols)

    # Get global measures
    over  = mtx.loc['overall_accuracy', mcols[0]]
    kappa = mtx.loc['kappa', mcols[0]]
    tarea = mtx.loc['total_area', 'row_area']
    mf1   = mtx.loc['macro-f1', mcols[0]]
    wf1   = mtx.loc['weighted-f1', mcols[0]]

    # Get f1-scores
    f1_scores = mtx.loc['f1-score', mcols].tolist()

    sintese.append([ename, tarea, over, kappa, mf1, wf1] + f1_scores)

    mtx.insert(0, 'class', mtx.index)

    mtxs.append(mtx)
    expname.append(ename)

    i += 1

# Save result
sdf = pd.DataFrame(sintese, columns=maincols)

alldfs = [sdf]

alldfs.extend(mtxs)
sheets = ['sintese'] + expname

obj_to_tbl(alldfs, os.path.join(out, 'val_table.xlsx'), sheetsName=sheets)